In [18]:
import numpy as np 
import pandas as pd 
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [82]:
def create_features(dataset_list):
    new_dataset_list = []
    for alarm, _ in dataset_list:
        # Add duration
        alarm['duration'] = alarm['end_timestamp'] - alarm['start_timestamp']
        
        # Add one-hot encoding
        alarm_one_hot = pd.get_dummies(alarm['alarm_id'], prefix='alarm')
        device_one_hot = pd.get_dummies(alarm['device_id'], prefix='device')

        # Time-windowed aggregations
        alarm['count_last_5min'] = alarm.groupby('device_id')['alarm_id'].transform(lambda x: x.rolling(window=300, min_periods=0).count()).astype(int)
        alarm['count_last_3min'] = alarm.groupby('device_id')['alarm_id'].transform(lambda x: x.rolling(window=180, min_periods=0).count()).astype(int)

        # Alarm sequences
        alarm['last_alarm'] = alarm.groupby('device_id')['alarm_id'].shift().fillna(-1).astype(int)  # Fill NaN with -1

        # Elapsed time
        alarm['time_since_last_alarm'] = alarm.groupby('device_id')['start_timestamp'].diff().fillna(-1).astype(int)  # Fill NaN with -1

        # Concatenate one-hot encoded features
        alarm = pd.concat([alarm, alarm_one_hot, device_one_hot], axis=1)

        new_dataset_list.append((alarm, _))
        
    return new_dataset_list

In [96]:
def my_train_test_split(dataset_list)
    counter = 0
    dataset_dict = {}

    for i in range(4):
        for j in range(4):
            if i != j:
                X_train, y_train = dataset_list[i][0]
                X_test, y_test = dataset_list[j][1]

                dataset_dict[f'dataset_{counter}'] = {
                    'X_train': X_train,
                    'y_train': y_train,
                    'X_test': X_test,
                    'y_test': y_test
                }

                counter += 1
        return dataset_dict

In [57]:
def train_model(X_train, y_train):
    # Model architecture
    # Model training
    return trained_model

In [59]:

def evaluate_model(trained_model, X_test, y_test):
    # Calculate metrics
    return accuracy, precision, recall

In [60]:
def fine_tune_model(trained_model):
    # Hyperparameter tuning
    return fine_tuned_model

In [61]:
def deploy_model(fine_tuned_model, new_data):
    # Make predictions
    return root_causes

In [65]:
alarm1 = pd.read_csv('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_1/alarm.csv')
causal1 = np.load('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_1/causal_prior.npy' , allow_pickle = True)

alarm2 = pd.read_csv('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_2/alarm.csv')
causal2 = np.load('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_2/causal_prior.npy' , allow_pickle = True)

alarm3 = pd.read_csv('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_3/alarm.csv')
causal3 = np.load('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_3/causal_prior.npy' , allow_pickle = True)

alarm4 = pd.read_csv('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_4/alarm.csv')
causal4 = np.load('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_4/causal_prior.npy' , allow_pickle = True)

rca1 = pd.read_csv('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_1/rca_prior.csv')
topology1 = np.load('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_1/topology.npy' , allow_pickle = True)
rca2 = pd.read_csv('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_2/rca_prior.csv')
topology2 = np.load('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_2/topology.npy' , allow_pickle = True)
rca3 = pd.read_csv('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_3/rca_prior.csv')
topology3 = np.load('/kaggle/input/causal-structure-learning-from-event-sequences/datasets/dataset_3/topology.npy' , allow_pickle = True)

In [92]:
dataset_list = [(alarm1, causal1), (alarm2, causal2), (alarm3, causal3), (alarm4, causal4)]
dataset_list = create_features(dataset_list)
split_dataset_dict = my_train_test_split(dataset_list)